# Módulo 2: Scraping con Selenium
## LATAM Airlines
<a href="https://www.latam.com/es_ar/"><img src="https://i.pinimg.com/originals/dd/52/74/dd5274702d1382d696caeb6e0f6980c5.png"  width="420"></img></a>
<br>

Vamos a scrapear el sitio de Latam para averiguar datos de vuelos en funcion el origen y destino, fecha y cabina. La información que esperamos obtener de cada vuelo es:
- Precio(s) disponibles
- Horas de salida y llegada (duración)
- Información de las escalas

**¡Empecemos!**
Utilicemos lo aprendido hasta ahora para lograr el objetivo propuesto

## Selenium

Selenium es una herramienta que nos permitirá controlar un navegador y podremos utilizar las funcionalidades del motor de JavaScript para cargar el contenido que no viene en el HTML de la página. Para esto necesitamos el módulo `webdriver`.

In [1]:
from selenium import webdriver
#importampos libreria para cargar el driver automaticamente
from webdriver_manager.firefox import GeckoDriverManager

url='https://www.latamairlines.com/py/es/ofertas-vuelos?origin=ASU&outbound=2023-08-01T12%3A00%3A00.000Z&destination=BCN&inbound=null&adt=1&chd=0&inf=0&trip=OW&cabin=Economy&redemption=false&sort=RECOMMENDED'

Necesitamos controladores web para diferentes navegadores web.<br>
Paso 1: instanciar un **driver** del navegador

In [2]:
options = webdriver.FirefoxOptions()
# Podemos agregarle opciones al driver para utilizar los distintos modos del navegador
options.add_argument('-private')
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install(), options=options)

[WDM] - Downloading: 19.0kB [00:00, 9.77MB/s]                   
C:\Users\mario\AppData\Local\Temp\ipykernel_17916\1250048064.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=GeckoDriverManager().install(), options=options)


Paso 2: hacer que el navegador cargue la página web.

In [3]:
driver.get(url)

paso 3:Extraer informacion de la pagina<br>
Carguemos la página y analicemos dónde se encuentra la información<br>
Vemos que el bloque de vuelos se encuentra en una `ul` y que cada vuelo es un item de la lista, `li`. <br>

In [4]:
#Usaremos el Xpath para obtener la lista de vuelos
vuelos = driver.find_elements('xpath','//ol/li')
print (vuelos)

[<selenium.webdriver.remote.webelement.WebElement (session="056ab3dd-c97e-42e1-98aa-3ce3f7ce5e61", element="730ecea6-e8c6-4442-8a0e-13d985970eea")>, <selenium.webdriver.remote.webelement.WebElement (session="056ab3dd-c97e-42e1-98aa-3ce3f7ce5e61", element="0ee30711-ddd3-435e-b79f-dc8106389470")>, <selenium.webdriver.remote.webelement.WebElement (session="056ab3dd-c97e-42e1-98aa-3ce3f7ce5e61", element="ed93d5c3-b927-4685-bfdf-894c7bafa861")>, <selenium.webdriver.remote.webelement.WebElement (session="056ab3dd-c97e-42e1-98aa-3ce3f7ce5e61", element="5552d710-5aca-4b50-a01f-3ae07d432257")>, <selenium.webdriver.remote.webelement.WebElement (session="056ab3dd-c97e-42e1-98aa-3ce3f7ce5e61", element="7cc2b57f-6cbc-480b-9345-c5a73d0777b0")>, <selenium.webdriver.remote.webelement.WebElement (session="056ab3dd-c97e-42e1-98aa-3ce3f7ce5e61", element="ab4dc3c4-0fa9-41d2-b6c8-a357b616e2e2")>, <selenium.webdriver.remote.webelement.WebElement (session="056ab3dd-c97e-42e1-98aa-3ce3f7ce5e61", element="3aed

Obtengamos la informacion de la hora de salida, llegada y duracion del vuelo

Notar que el xpath comienza con ".", lo que indica que sólo debe buscar en los hijos de ese elemento. Si no ponemos el ".", busca en todo el árbol.<br>
Elementos importantes de xpath:
- "//" busca en todos los hijos del elemento. "/" busca sólo en hijos directos
- "." indica que la búsqueda debe empezar en ese elemento y no en el origen del árbol
- Los atributos de los tags se buscan entre [] y con @

In [5]:
#seleccionamos el primer vuelo
vuelo_1=vuelos[0]
#hora de salida
hora_salida=vuelo_1.find_element('xpath','//div[@class="sc-klSiHT hjzFuR flight-information"]/span[1]').text
print (hora_salida)

6:55 a. m.


In [6]:
#hora de llegada
hora_llegada=vuelo_1.find_element('xpath','.//div[3]/span[1]').text.replace('\n+1','')
print (hora_llegada)

5:20 p. m.


In [7]:
# Duracion del vuelo
duracion_vuelo= vuelo_1.find_element('xpath','.//div[2]/span[2]').text
print (duracion_vuelo)

28 h 25 min


veremos cómo obtener la información de las escalas de cada vuelo. <br>
Para desplegar esa información de las escalas, debemos clickear en link para que se habilite el modal que contiene la informacion. Seleccionémoslo:

In [8]:
link_escalas = vuelo_1.find_element('xpath','//div[@class="sc-iKiVwC fbWfQZ"]//a')
print (link_escalas)

<selenium.webdriver.remote.webelement.WebElement (session="056ab3dd-c97e-42e1-98aa-3ce3f7ce5e61", element="952987df-ff2e-48d1-8261-2444e487a365")>


Una vez ubicado el elemento podemos clickear sobre él

In [9]:
link_escalas.click()

Y vemos cómo se despliega la información que estamos buscando enn el driver del navegador. **Notar que cambió el html de la página al hacer click sobre ese botón**

ahora debemos seleccionar los segmentos que contiene el vuelo para calcular la cantidad de escalas, actualmente vemos que las paradas estan contenidas en elementos sections, asi que debemos contabilizar las paradas

In [10]:
segmentos= link_escalas.find_elements('xpath','//section[@class="sc-fGSyRc fCuylQ"]')
# print(segmentos, len(segmentos))
for i in segmentos:
    print (i)

<selenium.webdriver.remote.webelement.WebElement (session="056ab3dd-c97e-42e1-98aa-3ce3f7ce5e61", element="d44b7177-1687-4966-9196-2312154eea0d")>
<selenium.webdriver.remote.webelement.WebElement (session="056ab3dd-c97e-42e1-98aa-3ce3f7ce5e61", element="66dcf5b8-6f35-4ce9-bc17-ff4260a23254")>
<selenium.webdriver.remote.webelement.WebElement (session="056ab3dd-c97e-42e1-98aa-3ce3f7ce5e61", element="ed22f711-8489-4264-9cf7-b754fc8fb396")>


para calcular las escalas, debemos restar -1 la cantidad total de paradas

In [11]:
#0 escalas si es un vuelo directo
escala=len(segmentos)-1
print(escala)

2


ahora obtengamos desde el modal el Itinerario de vuelo:
- origen ✔
- hora de salida 
- destino vuelo ✔
- hora de llegada
- duración del vuelo ✔
- duración de la escala. *Tip: el último segmento no tendrá esta información*
- número del vuelo
- modelo del avión

In [12]:
for i in segmentos:
    segmento = i
    salida=segmento.find_element('xpath','.//div[@class="sc-jFpLkX jAGOAr"]/div[@class="sc-fguZLD kepXur"]/div[@class="iataCode"]/span[1]').text
    hora_salida=segmento.find_element('xpath','.//div[@class="sc-jFpLkX jAGOAr"]/div[@class="sc-fguZLD kepXur"]/div[@class="iataCode"]/span[2]').text
    duracion_segmento=segmento.find_element('xpath','.//div[@class="sc-jFpLkX jAGOAr"]//div[@class="sc-ewMkZo hQNSAX"]/span[2]').text
    destino_segmento=segmento.find_element('xpath','.//div[@class="sc-jFpLkX jAGOAr"]//div[@class="sc-eCXBzT goeYBu"]/div[@class="iataCode"]/span[1]').text
    numero_vuelo_segmento=segmento.find_element('xpath','.//div[@class="sc-dzQEYZ dslPlz airline-wrapper"]').text
    modelo_avion_segmento=segmento.find_element('xpath','.//div[@class="sc-sVRsr eXYUTi"]//span[@class="airplane-code"]').text
    
    print(f'Salida:{salida}\nHora Salida:{hora_salida}\nDuracion: {duracion_segmento}\nLlegada:{destino_segmento}\nNumero de vuelo:{numero_vuelo_segmento}\nModelo Avion:{modelo_avion_segmento}\n')


Salida:ASU
Hora Salida:6:55 a. m.
Duracion: 4 h 5 min
Llegada:LIM
Numero de vuelo:LA1320
Modelo Avion:Airbus A320

Salida:LIM
Hora Salida:8:20 p. m.
Duracion: 11 h 15 min
Llegada:MAD
Numero de vuelo:LA5391
Modelo Avion:Airbus A350-900

Salida:MAD
Hora Salida:4:00 p. m.
Duracion: 1 h 20 min
Llegada:BCN
Numero de vuelo:LA1523
Modelo Avion:Airbus A320



In [13]:
escalas_vuelo=link_escalas.find_elements('xpath','//section[@class="sc-kiXyGy sc-eZXMBi dKgCnQ connectionInfo"]')
for i in escalas_vuelo:
    escala=i
    escala_vuelo=escala.find_element('xpath','.//div[@class="sc-ekQYnd cByWfv"]//span[@class="connection-text"]').text
    duracion_escala_vuelo=escala.find_element('xpath','.//div[@class="sc-ekQYnd cByWfv"]//span[@class="time"]').text
    print(f'Escala:{escala_vuelo}\nDuracion Escala:{duracion_escala_vuelo}')
    # print (i)

Escala:Conexión Lima
Duracion Escala:10 h 20 min
Escala:Conexión Madrid
Duracion Escala:1 h 25 min


Una vez que hayamos obtenido toda la información, debemos cerrar el modal/pop-up.

In [14]:
driver.find_element('xpath','//*[@id="itinerary-modal-0-dialog-close"]').click()

Por último debemos obtener la información de las tarifas. Para eso, debemos clickear sobre el vuelo (sobre cualquier parte)

In [15]:
vuelo_1.click()

La información de los precios para cada tarifa está contenida en una tablas (ol). Tenemos los precios y categorias de cada una de las tarifas

In [16]:
tarifas= vuelo_1.find_elements('xpath','.//ol[@class="sc-buGlAa jhwXGF"]/li[@class="sc-kecUPG dPNrrD"]')
print (tarifas)

[<selenium.webdriver.remote.webelement.WebElement (session="056ab3dd-c97e-42e1-98aa-3ce3f7ce5e61", element="b920dd40-4600-42cd-925b-faec9d5074b8")>, <selenium.webdriver.remote.webelement.WebElement (session="056ab3dd-c97e-42e1-98aa-3ce3f7ce5e61", element="4200726a-fafc-4c45-968b-19d6392d2ee2")>]


In [17]:
# creamos una lista donde almacenaremos los precios de cada tarifa
precios=[]
for tarifa in tarifas:
    #buscamos en cada pocision de la tarifa los siquientes elementos
    nombre = tarifa.find_element('xpath','.//div[@class="sc-gGsJSs dhstcp"]/div[1]/span[@class="sc-fhiYOA iwcbaW"]').text
    moneda= tarifa.find_element('xpath','.//div[@class="sc-gGsJSs dhstcp"]/div[3]//span[contains(@class,"currency")]').text
    valor= tarifa.find_element('xpath','.//div[@class="sc-gGsJSs dhstcp"]/div[3]//span[@class="sc-ckYZGd grNCid"]').text
    #guardo los valores que obtengo en un diccionario
    dict_tarifa={nombre:{'moneda':moneda,'valor':valor}}
    #guradamos nuestro diccionario con los datos de tarifa a nuestra lia de precio
    precios.append(dict_tarifa)
    print(dict_tarifa)
    # print(f'Nombre Tarfia:{nombre}\nMoneda:{moneda}\nPrecio:{valor}\n\n')

{'plus': {'moneda': 'USD', 'valor': '1,341.20'}}
{'top': {'moneda': 'USD', 'valor': '3,153.20'}}


Paso 4: cerrar el navegador

In [18]:
driver.close()